# 🫡 Save the Model

We need to save this model so that we can use it from various locations, including other notebooks or the model server, so we upload it to an s3-compatible storage.

>NOTE: Don't run all the cells all-in-one shot without changing the cluster specific variables.

### Install the required packages and define a function for the upload

If `pip` gives an error, don't worry about it. Things will just run fine regardless.

In [ ]:
!pip -q install model-registry==0.2.15

# 🤩 Kubeflow Registry

We need a metadata registry for storing information such as version, author, and model location of the models we are building.

We are using Kubeflow model registry as a canonical data source by storing such information.

Here are some reasons to use a registry (_from Kubeflow website_):

- Track models available on storage: once the model is stored, it can then be tracked in the Kubeflow Model Registry for managing its lifecycle. The Model Registry can catalog, list, index, share, record, organize this information. This allows the Data Scientist to compare different versions and revert to previous versions if needed.

- Track and compare performance: View key metrics like accuracy, recall, and precision for each model version. This helps identify the best-performing model for deployment.

- Create lineage: Capture the relationships between data, code, and models. This enables the Data Scientist to understand the origin of each model and reproduce specific experiments.

- Collaborate: Share models and experiment details with the MLOps Engineer for deployment preparation. This ensures a seamless transition from training to production.

An instance of the registry is available in your dev environment as well. 


# 🪣 S3 Storage

We use S3 storage as the backend for our Model Registry. This means our models are stored in S3, and the Model Registry keeps track of their locations. 

Thanks to the Kubeflow Model Registry, we can push our models to both our S3 storage and register them at the same time, making it easy to store and keep track of new models we produce 🕵️

In [ ]:
from model_registry import ModelRegistry
from model_registry.utils import S3Params
from model_registry.exceptions import StoreError
import os

‼️⚠️ IMPORTANT ⚠️‼️

Add your user name and cluster domain (apps.xxx) that are shared with you before. We need them for the model registry URL.

In [ ]:
# Add your user name and cluster domain (apps.xxx) that are shared with you before

username = "<USER_NAME>"
cluster_domain = "<CLUSTER_DOMAIN>"

In [ ]:
# Set up the model registry connection
model_registry_url = f"https://{username}-registry-rest.{cluster_domain}"
author_name = username

registry = ModelRegistry(server_address=model_registry_url, port=443, author=author_name, is_secure=False)

In [ ]:
# Model details we want to register
registered_model_name = "jukebox"
version = "0.0.1"
model_path = "models/jukebox/"

#### 🙏 Thanks to data connections, S3 bucket credentials are available in the notebook!

We explicitly fetch the S3 bucket, but the others are used automagically behind the scenes when we upload and register our model 🧙‍♂️

In [ ]:
s3_upload_params = S3Params(
    bucket_name=os.environ.get('AWS_S3_BUCKET'),
    s3_prefix="models/jukebox/",
)

try:
    registered_model = registry.upload_artifact_and_register_model(
        name=registered_model_name,
        model_files_path=model_path,
        model_format_name="onnx",
        author=username,
        model_format_version="1",
        version=version,
        description="Dense Neural Network trained on music data",
        metadata={
            "accuracy": 0.3,
            "license": "apache-2.0"
        },
        upload_params=s3_upload_params
    )

    print(f"'{registered_model_name}' version '{version}' has been registered here: https://rhods-dashboard-redhat-ods-applications.{cluster_domain}/modelRegistry/{username}-registry/registeredModels/1/versions/{registry.get_model_version(registered_model_name, version).id}/details")

except StoreError:
    rmver = registry.get_model_version(registered_model_name, version)
    print(f"Model and version already exists:\n{rmver}")

### Quiz Time 🤓

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../.dontlookhere/'))
try: from quiz2 import *
except: pass

In [ ]:
try: quiz_versioning()
except: pass

### We can also get some nice information about our model from the Model Registry

In [ ]:
# Print the general info of registered model
model = registry.get_registered_model("jukebox")
print("Registered Model:", model, "with ID", model.id)

In [ ]:
# Print the version info of registered model
version = registry.get_model_version("jukebox", "0.0.1")
print("Model Version:", version, "with ID", version.id)

In [ ]:
# Print the artifact info of registered model
art = registry.get_model_artifact("jukebox", "0.0.1")
print("Model Artifact:", art, "with ID", art.id)

### 🥁 Next Step

Now that you've saved the model to S3 storage & registry, you can refer to the model by using a data connection and serve the model as an API.

Go back to the instructions https://rhoai-mlops.github.io/lab-instructions/#/1-when-the-music-starts/4-inner-data-science-loop?id=model-serving to view the model in Model Registry UI first.